In [2]:
import scanpy as sc
import pandas as pd
import os
import anndata as ad
import scipy as sp
from tqdm import tqdm

In [2]:
df_anno = pd.read_csv('../../data/GSE195452/GSE195452_Cell_metadata_v26_anno.txt',
                      sep="\t")

/tmp/ipykernel_193527/2914917506.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_anno = pd.read_csv('../../data/GSE195452/GSE195452_Cell_metadata_v26_anno.txt',


In [3]:
df_anno.head()

,Well_ID,well_coordinates,Amp_batch_ID,Cell_barcode,Seq_batch_ID,Pool_barcode,Pool_barcode_i5,Number_of_cells,annotation
WMC2588321,WMC2588321,A1,AB10128,CTATTCG,SB400,GTCACTGT,TAACGGCT,1,_
WMC2588322,WMC2588322,C1,AB10128,ACTCAGG,SB400,GTCACTGT,TAACGGCT,1,_
WMC2588323,WMC2588323,E1,AB10128,ACACGTG,SB400,GTCACTGT,TAACGGCT,1,Peri_RGS5
WMC2588324,WMC2588324,G1,AB10128,TCTTCGA,SB400,GTCACTGT,TAACGGCT,1,Fibro_MYOC1
WMC2588325,WMC2588325,I1,AB10128,AGCACTG,SB400,GTCACTGT,TAACGGCT,1,_


In [4]:
df2 = pd.read_csv('../../data/GSE195452/SraRunTable.txt')

In [5]:
# read counts
directory = '../../data/GSE195452/rna'
adata_list = list()

for root, dirs, files in os.walk(directory):
    for filename in tqdm(files):
        df = pd.read_csv(os.path.join(root, filename), sep='\t').transpose()
        
        counts = sp.sparse.csr_matrix(df.to_numpy())
        
        adata = ad.AnnData(counts)
        adata.obs_names = df.index.values.tolist()
        adata.var_names = df.columns.values.tolist()
        
        adata.obs = df_anno.loc[adata.obs_names, ]
        
        sample_name = filename.split("_")[0]
        sample_info = df2[df2['Sample Name'] == sample_name]
        
        adata.obs['sample_name'] = sample_name
        adata.obs['selection_marker'] = sample_info['selection_marker'].values.tolist()[0]
        adata.obs['source_name'] = sample_info['source_name'].values.tolist()[0]
        adata.obs['tissue'] = sample_info['Tissue'].values.tolist()[0]
        adata.obs['patient_id'] = sample_info['PATIENT_ID'].values.tolist()[0]
        adata.obs['organism'] = sample_info['Organism'].values.tolist()[0]
        
        adata_list.append(adata)

100%|██████████| 723/723 [12:04<00:00,  1.00s/it]


In [6]:
adata = ad.concat(adata_list, label="batch", index_unique="_")

In [7]:
# filter cells by annotation
adata = adata[adata.obs['annotation'] != '_']

In [8]:
adata.write_h5ad('../../results/cell_2022/scrna.h5ad')

/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/zl_spatial/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/zl_spatial/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/zl_spatial/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/zl_spatial/lib/python3.10/site-packages/anndata/_core/anndata.py:1222: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/z

In [9]:
adata

AnnData object with n_obs × n_vars = 147836 × 57874
    obs: 'Well_ID', 'well_coordinates', 'Amp_batch_ID', 'Cell_barcode', 'Seq_batch_ID', 'Pool_barcode', 'Pool_barcode_i5', 'Number_of_cells', 'annotation', 'sample_name', 'selection_marker', 'source_name', 'tissue', 'patient_id', 'organism', 'batch'